### 100% Opensource Solutions 

In [3]:
! pip install mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 7.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.9.0.post0
    Uninstalling python-dateutil-2.9.0.post0:
      Successfully uninstalled python-dateutil-2.9.0.post0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.10.1
    Uninstalling pydantic_core-2.10.1:
      Successfully uninstalled pydantic_core-2.10.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.4.2
    Uninstalling pydantic-2.4.2:
      Successfully uninstalled pydantic-2.4.2
ERROR: pip's dependency r

In [ ]:
import time
import pandas as pd
from mistralai import Mistral

In [16]:
api_key = ""
questions_file = "/Hackaton/data/questions.csv"

df = pd.read_csv(questions_file)

### SOLUTION 1: Prompt engineering

In [ ]:
import time
import pandas as pd
from tqdm import tqdm
from mistralai import Mistral  # Assuming this is your client
import random

question_prompt = lambda body, possible_answer_a, possible_answer_b, possible_answer_c, possible_answer_d, possible_answer_e: (
    "Vous êtes un expert médical chargé de répondre à la question à choix multiples suivante. La question peut avoir une ou plusieurs réponses correctes. "
    "Votre objectif est de fournir la réponse la plus précise en vous basant sur vos connaissances médicales approfondies et votre compréhension du sujet. "
    "Veuillez lire attentivement et analyser la question et toutes les réponses possibles avant de répondre. "
    "Utilisez votre pensée critique et appliquez les principes médicaux pertinents pour déterminer quelles options sont correctes. "
    "Votre réponse doit consister **uniquement** en les lettres correspondant aux choix de réponses corrects, **en majuscules**, et séparées par des virgules sans espaces ni caractères supplémentaires. "
    "Par exemple:\n"
    "- Si les options A et C sont correctes, votre réponse doit être formatée exactement comme 'A,C'.\n"
    "- Si seule l'option D est correcte, votre réponse doit être 'D'.\n"
    "- Si toutes les options sont correctes, votre réponse doit être 'A,B,C,D,E'.\n"
    "- Si aucune des options n'est correcte et que la question le permet, votre réponse doit être 'Aucune'.\n"
    "N'**incluez** pas d'explications, de raisonnements ou de texte supplémentaire dans votre réponse. "
    "Ne répétez pas la question ou les choix de réponses. "
    "Assurez-vous que les lettres sont **en majuscules** et classées par **ordre alphabétique**. "
    "N'incluez pas de points, de parenthèses ou d'autres signes de ponctuation en dehors des virgules pour séparer les lettres. "
    "Votre réponse doit être concise et respecter strictement le format spécifié. "
    "Ne mentionnez **pas** d'incertitudes ou n'exprimez pas de doutes dans votre réponse. "
    "Fournissez uniquement la réponse définitive basée sur les informations données. "
    "Si la question précise qu'il y a au moins une réponse correcte, supposez qu'au moins une option est correcte même si vous pensez le contraire en fonction de vos connaissances. "
    "En cas d'informations contradictoires, fiez-vous aux connaissances médicales standard jusqu'à votre date de connaissance en 2023.\n\n"
    f"Question:\n{body}\n"
    f"A) {possible_answer_a}\n"
    f"B) {possible_answer_b}\n"
    f"C) {possible_answer_c}\n"
    f"D) {possible_answer_d}\n"
    f"E) {possible_answer_e}\n"
)

# Assuming df is your dataframe with the questions
answers = []
client = Mistral(api_key=api_key)

# Exponential backoff parameters
max_retries = 10  # Maximum number of retries
base_sleep_time = 2  # Starting sleep time for exponential backoff

batch_size = 10  # Process in batches of 10 rows
total_batches = len(df) // batch_size + (1 if len(df) % batch_size != 0 else 0)

# Iterate over batches of 10 rows
for batch_num in range(total_batches):
    start_idx = batch_num * batch_size
    end_idx = min(start_idx + batch_size, len(df))

    print(f"Processing batch {batch_num + 1}/{total_batches} (rows {start_idx} to {end_idx - 1})")
    
    for row_idx, row in tqdm(df.iloc[start_idx:end_idx].iterrows(), total=(end_idx - start_idx), desc=f"Processing batch {batch_num + 1}"):
        prompt = question_prompt(
            row["question"],
            row["answer_A"],
            row["answer_B"],
            row["answer_C"],
            row["answer_D"],
            row["answer_E"]
        )

        retries = 0
        success = False
        while not success and retries < max_retries:
            try:
                chat_response = client.chat.complete(
                    model="open-mistral-nemo",
                    messages=[
                        {
                            "role": "user",
                            "content": prompt
                        },
                    ],
                    temperature=0.1
                )
                answers.append(chat_response.choices[0].message.content)
                success = True  # Exit the retry loop on success
            except Exception as e:
                if '429' in str(e):  # Check for rate limit error
                    wait_time = base_sleep_time * (2 ** retries) + random.uniform(0, 1)  # Exponential backoff with jitter
                    print(f"Rate limit hit. Sleeping for {wait_time:.2f} seconds... (Retry {retries + 1}/{max_retries})")
                    time.sleep(wait_time)
                    retries += 1
                else:
                    print(f"Error on row {row_idx}: {e}")
                    answers.append(None)
                    break

        # If after max retries we are still failing, log the failure
        if retries == max_retries and not success:
            print(f"Max retries reached for row {row_idx}. Skipping...")

        # Sleep for 2 seconds after each iteration
        time.sleep(2)

    # After processing each batch of 10 rows, wait for 15 seconds
    if (batch_num + 1) < total_batches:
        print(f"Batch {batch_num + 1} completed. Sleeping for 15 seconds before the next batch...")
        time.sleep(15)

# Output format is a 2-column dataframe with the answers
output_df = pd.DataFrame(answers, columns=["Answer"])
output_df.index.name = "id"

# Save the result to CSV
output_df.to_csv("output_with_batches.csv")

In [ ]:
print(output_df.shape)

(101, 1)


### SOLUTION 2: WITH WIKIPEDIA API AND PROMPT ENGINEERING

In [ ]:
import time
import pandas as pd
from tqdm import tqdm
from mistralai import Mistral
import random
import requests

In [107]:
# Step 1: First prompt to transform the question into a search-friendly query (in French)
def transform_question_to_search_query(question):
    prompt = (
        "Vous êtes un expert médical chargé d'extraire le terme médical central ou la maladie principale à partir de la question ou de l'étude de cas suivante. "
        "Votre objectif est d'identifier un seul terme médical ou une seule condition médicale qui reflète le sujet central de la question, en excluant les adjectifs, contextes, âges, ou descriptions supplémentaires. "
        "Par exemple, si la question contient des informations supplémentaires comme 'fébrile' ou 'chez l'enfant', elles ne doivent pas être incluses dans le terme médical extrait. "
        "Concentrez-vous uniquement sur le terme médical principal (comme une maladie, un symptôme, ou un diagnostic spécifique) et non sur les détails contextuels.\n\n"
        f"Question ou étude de cas :\n{question}\n\n"
        "Veuillez fournir uniquement le terme ou la condition médicale principale sous la forme d'un seul mot ou d'une seule phrase concise."
    )
    return prompt


def generate_answer_from_wikipedia(wiki_results, question, possible_answers):
    # Utiliser les deux premiers résultats de Wikipédia
    if len(wiki_results) >= 2:
        wiki_info_1 = wiki_results[0]
        wiki_info_2 = wiki_results[1]
    elif len(wiki_results) == 1:
        wiki_info_1 = wiki_results[0]
        wiki_info_2 = ""
    else:
        wiki_info_1 = "Aucune information pertinente trouvée."
        wiki_info_2 = ""
    
    # Construire le prompt spécialisé
    prompt = (
        "Vous êtes un expert médical spécialisé en pédiatrie et maladies infectieuses. "
        "Votre tâche est d'analyser minutieusement chaque option de réponse pour la question ci-dessous, en vous basant sur les informations fournies de Wikipédia et vos connaissances médicales approfondies. "
        "Pour chaque option, évaluez sa validité en lien direct avec la question, en considérant les éléments clés tels que les symptômes, les causes, l'âge du patient, et la présentation clinique. "
        "Fournissez une justification détaillée pour expliquer pourquoi l'option est correcte ou incorrecte.\n\n"
        f"**Question :**\n{question}\n\n"
        "**Options de réponse :**\n"
    )
    
    # Ajouter les réponses possibles au prompt
    for idx, answer in enumerate(possible_answers):
        option_letter = chr(65 + idx)  # Convertit 0 en 'A', 1 en 'B', etc.
        prompt += f"{option_letter}) {answer}\n"
    
    prompt += (
        "\n**Informations de Wikipédia :**\n"
        f"Article 1:\n{wiki_info_1}\n\n"
        f"Article 2:\n{wiki_info_2}\n\n"
        "Pour chaque option, suivez les instructions suivantes :\n"
        "1. **Déterminez** si l'option est correcte ou incorrecte en relation avec la question posée.\n"
        "2. **Expliquez** votre raisonnement en détaillant les points pertinents, en vous référant spécifiquement aux informations de Wikipédia et à vos connaissances médicales.\n"
        "3. **Mettez en évidence** les éléments clés de la question qui influencent votre décision (par exemple, les symptômes spécifiques, le contexte clinique, l'âge du patient).\n"
        "4. **Soyez concis** mais exhaustif dans votre explication.\n\n"
        "Présentez votre analyse sous forme de liste, en commençant chaque point par la lettre de l'option correspondante suivie de \"Correct\" ou \"Incorrect\", puis de votre explication détaillée.\n\n"
        "### Exemple de format attendu :\n"
        "A) Correct. [Votre explication détaillée]\n"
        "B) Incorrect. [Votre explication détaillée]\n\n"
        "**Commencez votre analyse ci-dessous :**"
    )
    return prompt



# Step 3: Third prompt to choose the correct answer based on the second call output
def choose_correct_answer(question, possible_answer_a, possible_answer_b, possible_answer_c, possible_answer_d, possible_answer_e, second_call_output):
    prompt = (
        "Vous êtes un expert médical chargé de répondre à la question à choix multiples suivante. La question peut avoir une ou plusieurs réponses correctes. "
        "Votre objectif est de fournir la réponse la plus précise en vous basant sur vos connaissances médicales approfondies et les informations fournies dans la réponse générée lors du deuxième appel. "
        "Veuillez lire attentivement et analyser la question, les informations de la deuxième réponse, et toutes les réponses possibles avant de répondre. "
        "Utilisez les informations données et appliquez les principes médicaux pertinents pour déterminer quelles options sont les plus susceptibles d'être correctes. "
        "Votre réponse doit consister **uniquement** en les lettres correspondant aux choix de réponses les plus probables, **en majuscules**, et séparées par des virgules sans espaces ni caractères supplémentaires. "
        "Par exemple:\n"
        "- Si les options A et C sont correctes, votre réponse doit être formatée exactement comme 'A,C'.\n"
        "- Si seule l'option D est correcte, votre réponse doit être 'D'.\n"
        "- Si toutes les options sont correctes, votre réponse doit être 'A,B,C,D,E'.\n"
        "- Si aucune des options n'est correcte et que la question le permet, votre réponse doit être 'Aucune'.\n"
        "N'**incluez** pas d'explications, de raisonnements ou de texte supplémentaire dans votre réponse. "
        "Ne répétez pas la question ou les choix de réponses. "
        "Assurez-vous que les lettres sont **en majuscules** et classées par **ordre alphabétique**. "
        "N'incluez pas de points, de parenthèses ou d'autres signes de ponctuation en dehors des virgules pour séparer les lettres. "
        "Votre réponse doit être concise et respecter strictement le format spécifié. "
        "Ne mentionnez **pas** d'incertitudes ou n'exprimez pas de doutes dans votre réponse. "
        "Fournissez uniquement la réponse définitive basée sur les informations données et sur la réponse générée lors du deuxième appel.\n\n"
        f"Question :\n{question}\n\n"
        f"Informations de la réponse générée lors du deuxième appel :\n{second_call_output}\n\n"
        f"A) {possible_answer_a}\n"
        f"B) {possible_answer_b}\n"
        f"C) {possible_answer_c}\n"
        f"D) {possible_answer_d}\n"
        f"E) {possible_answer_e}\n\n"
        "Veuillez fournir uniquement les lettres correspondant à la bonne réponse."
    )
    return prompt


def fallback_to_model_knowledge(question):
    prompt = (
        "Vous êtes un expert médical chargé de répondre à la question suivante en utilisant uniquement vos connaissances médicales. "
        "Aucune information externe n'est disponible. "
        "Votre objectif est de fournir une réponse complète et précise en vous basant uniquement sur vos connaissances médicales."
        f"Question : {question}\n\n"
        "Veuillez fournir une réponse complète et concise."
    )
    return prompt

In [ ]:
# Function to search Wikipedia with error handling (returns the snippets of the first two results)
def search_wikipedia(query):
    url = "https://fr.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if 'query' in data and 'search' in data['query']:
            search_results = data['query']['search']
            if search_results:
                # Return the snippets of the first two results
                snippets = []
                for result in search_results[:2]:
                    snippets.append(result['snippet'])
                return snippets
            else:
                print(f"Aucun résultat trouvé pour la requête : {query}")
                return None
        
        else:
            print(f"Aucun résultat trouvé pour la requête : {query}")
            return None
    
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la recherche sur Wikipédia : {e}")
        return None



In [109]:

answers = []
client = Mistral(api_key=api_key)

# Parameters for exponential backoff retry
max_retries = 10  # Maximum number of retries
base_sleep_time = 2  # Base sleep time for exponential backoff (reduced)
long_wait_time = 5  # Wait time in case of rate limit error (reduced)
jitter_factor = 3  # Jitter factor for randomness in sleep

# Process the first five rows
df_first_five_rows = df.iloc[:5]  # Process the first five rows

# Iterate over the first five rows
for row_idx, row in tqdm(df_first_five_rows.iterrows(), total=5, desc="Traitement des cinq premières lignes"):
    question = row["question"]
    
    # Step 1: Transform the question into a search query using the first prompt
    search_query_prompt = transform_question_to_search_query(question)
    
    # Call the model to generate a search query
    retries = 0
    success = False
    while not success and retries < max_retries:
        try:
            search_query_response = client.chat.complete(
                model="open-mixtral-8x22b",
                messages=[
                    {
                        "role": "user",
                        "content": search_query_prompt
                    },
                ],
                temperature=0.1
            )
            search_query = search_query_response.choices[0].message.content.strip()
            print(f"Requête de recherche générée : {search_query}")
            success = True
        except Exception as e:
            if '429' in str(e):
                wait_time = long_wait_time
                print(f"Limite de taux atteinte. Pause pendant {wait_time} secondes... (Tentative {retries + 1}/{max_retries})")
                time.sleep(wait_time)
                retries += 1
            else:
                print(f"Erreur lors de la génération de requête pour la ligne {row_idx} : {e}")
                break
    
    # Reduced pause after the first call
    time.sleep(base_sleep_time)
    
    # Step 2: Use the search query to get the first two Wikipedia results
    wiki_results = search_wikipedia(search_query)
    if wiki_results:
        print(f"Extraits de Wikipédia trouvés : {wiki_results}")
    else:
        print(f"Aucun résultat pertinent trouvé pour la requête : {search_query}")
    
    # Verify if the Wikipedia results are valid, otherwise use a fallback
    if not wiki_results:
        print(f"Utilisation des connaissances du modèle pour la ligne {row_idx}")
        generate_answer_prompt = fallback_to_model_knowledge(question)
    else:
        # Prepare the possible answers list
        possible_answers = [
            row["answer_A"],
            row["answer_B"],
            row["answer_C"],
            row["answer_D"],
            row["answer_E"]
        ]
        generate_answer_prompt = generate_answer_from_wikipedia(wiki_results, question, possible_answers)
    
    print(f"Prompt envoyé au modèle : {generate_answer_prompt}")
    
    # Step 3: Generate a complete answer based on the Wikipedia results or fallback to internal knowledge
    generated_answer = None
    retries = 0
    success = False
    while not success and retries < max_retries:
        try:
            second_call_response = client.chat.complete(
                model="open-mixtral-8x22b",
                messages=[
                    {
                        "role": "user",
                        "content": generate_answer_prompt
                    },
                ],
                temperature=0.2
            )
            if second_call_response and second_call_response.choices:
                generated_answer = second_call_response.choices[0].message.content.strip()
                print(f"Réponse complète générée : {generated_answer}")
                success = True
            else:
                print(f"Aucune réponse valide reçue pour la ligne {row_idx}")
                break
        except Exception as e:
            if '429' in str(e):
                wait_time = long_wait_time
                print(f"Limite de taux atteinte. Pause pendant {wait_time} secondes... (Tentative {retries + 1}/{max_retries})")
                time.sleep(wait_time)
                retries += 1
            else:
                print(f"Erreur sur la génération de réponse pour la ligne {row_idx} : {e}")
                break
    
    # Reduced pause after the second call
    time.sleep(base_sleep_time)
    
    # Step 4: Use the generated answer to choose the correct multiple-choice option
    if generated_answer:
        prompt_third_call = choose_correct_answer(
            question,
            row["answer_A"],
            row["answer_B"],
            row["answer_C"],
            row["answer_D"],
            row["answer_E"],
            second_call_output=generated_answer
        )
        
        retries = 0
        success = False
        while not success and retries < max_retries:
            try:
                third_call_response = client.chat.complete(
                    model="open-mixtral-8x22b",
                    messages=[
                        {
                            "role": "user",
                            "content": prompt_third_call
                        },
                    ],
                    temperature=0.1
                )
                if third_call_response and third_call_response.choices:
                    answer = third_call_response.choices[0].message.content.strip()
                    print(f"Réponse sélectionnée : {answer}")
                    answers.append(answer)
                    success = True
                else:
                    print(f"Aucune réponse valide reçue pour la ligne {row_idx}")
                    break
            except Exception as e:
                if '429' in str(e):
                    wait_time = long_wait_time
                    print(f"Limite de taux atteinte. Pause pendant {wait_time} secondes... (Tentative {retries + 1}/{max_retries})")
                    time.sleep(wait_time)
                    retries += 1
                else:
                    print(f"Erreur sur la ligne {row_idx} : {e}")
                    answers.append(None)
                    break
        
        # If after max retries we are still failing, log the failure
        if retries == max_retries and not success:
            print(f"Tentatives maximales atteintes pour la ligne {row_idx}. Passage à la ligne suivante...")
        
        # Reduced pause after the third call
        time.sleep(base_sleep_time)
    else:
        print(f"Réponse générée manquante pour la ligne {row_idx}, question : {question}. Passage à la ligne suivante...")
        answers.append(None)
        
# Output format is a DataFrame with the answers
output_df = pd.DataFrame(answers, columns=["Answer"])
output_df.index.name = "id"

# Save the result to CSV
output_df.to_csv("output_first_five_rows.csv")


Traitement des cinq premières lignes:   0%|          | 0/5 [00:00<?, ?it/s]

Requête de recherche générée : Exanthème roséoliforme
Extraits de Wikipédia trouvés : ['projets correspondants. Pour un article plus général, voir <span class="searchmatch">exanthème</span>. <span class="searchmatch">L’exanthème</span> <span class="searchmatch">roséoliforme</span> est une éruption cutanée érythémateuse brutale et transitoire', 'secondaire\xa0; <span class="searchmatch">exanthème</span> morbilliforme\xa0: maculo-papules rouges pouvant confluer en plaques séparées par des intervalles de peau saine\xa0; <span class="searchmatch">exanthème</span> <span class="searchmatch">roséoliforme</span>\xa0: petites']
Prompt envoyé au modèle : Vous êtes un expert médical spécialisé en pédiatrie et maladies infectieuses. Votre tâche est d'analyser minutieusement chaque option de réponse pour la question ci-dessous, en vous basant sur les informations fournies de Wikipédia et vos connaissances médicales approfondies. Pour chaque option, évaluez sa validité en lien direct avec la questio

Traitement des cinq premières lignes:  20%|██        | 1/5 [00:14<00:57, 14.28s/it]

Requête de recherche générée : Insuffisance cardiaque
Extraits de Wikipédia trouvés : ['Pour les articles homonymes, voir IC. <span class="searchmatch">Insuffisance</span> <span class="searchmatch">cardiaque</span> Signes cliniques et symptômes de <span class="searchmatch">l\'insuffisance</span> <span class="searchmatch">cardiaque</span>. Mise en garde médicale modifier -', 'terme valvulopathie <span class="searchmatch">cardiaque</span> (littéralement, maladie des valves <span class="searchmatch">cardiaques</span>) désigne divers dysfonctionnements des valves <span class="searchmatch">cardiaques</span>. Ce sont des maladies']
Prompt envoyé au modèle : Vous êtes un expert médical spécialisé en pédiatrie et maladies infectieuses. Votre tâche est d'analyser minutieusement chaque option de réponse pour la question ci-dessous, en vous basant sur les informations fournies de Wikipédia et vos connaissances médicales approfondies. Pour chaque option, évaluez sa validité en lien direct avec la 

Traitement des cinq premières lignes:  20%|██        | 1/5 [00:23<01:34, 23.73s/it]


KeyboardInterrupt: 

In [45]:
print(output_df)

Empty DataFrame
Columns: [Réponse]
Index: []
